## Trabajo Práctico 1 :

Clasificador de Recomendaciones Recreativas utilizando Procesamiento de Lenguaje Natural (NLP)

Integrantes del grupo:

*   Dalmau, Valentín.
*   Nardi, Gianella.

### 1. Clasificación del Estado de Ánimo:

#### Importacion de Librerías

In [ ]:
!pip install spacy
!python -m spacy download xx_ent_wiki_sm
!pip install transformers sentence_transformers
!pip install requests beautifulsoup4
!pip install gliner
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
import nltk
from sklearn.pipeline import make_pipeline
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import requests
from bs4 import BeautifulSoup
from google.colab import files
from sklearn.model_selection import cross_val_score
from gliner import GLiNER
import unicodedata
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 53.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('xx_ent_wiki_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


#### Etiquetas y datasets para entrenar el modelo

In [ ]:
# Descargamos los stopwords que necesitaremos luego
nltk.download('stopwords')
from nltk.corpus import stopwords

# Obtenemos las stopwords para español
spanish_stop_words = stopwords.words('spanish')

# Definimos las etiquetas de estados de ánimo
labels = [(0, "Triste"), (1, "Melancólico"), (2, "Neutral"), (3, "Alegre"), (4, "Eufórico")]

# Dataset con ejemplos para cada estado de ánimo
dataset = []

# Textos que representan el estado de ánimo "Triste"
dataset.extend([
    (0, "Nada parece tener sentido últimamente."),
    (0, "Cada día se siente como una carga difícil de llevar."),
    (0, "Siento que nada mejora, todo es igual de oscuro."),
    (0, "Es un peso en el pecho que no puedo quitarme."),
    (0, "Los días pasan y la tristeza no desaparece."),
    (0, "No encuentro la motivación para hacer nada."),
    (0, "Mis pensamientos son negativos y difíciles de controlar."),
    (0, "Nada parece traerme alegría, todo está en sombras."),
    (0, "La tristeza me envuelve y no veo la salida."),
    (0, "Apenas puedo concentrarme, todo se siente borroso."),
    (0, "El vacío es constante, me siento sin rumbo."),
    (0, "Mis ganas de seguir adelante se desvanecen."),
    (0, "El dolor en el corazón es constante y profundo."),
    (0, "Nada parece capaz de mejorar mi estado."),
    (0, "Es como si estuviera en una espiral de oscuridad."),
    (0, "No encuentro fuerzas para salir de esta situación."),
    (0, "No tengo interés en nada ni en nadie."),
    (0, "Es difícil recordar la última vez que sonreí."),
    (0, "La tristeza es una carga que llevo todo el tiempo."),
    (0, "Los días parecen interminables y vacíos."),
    (0, "Es un peso que no desaparece, cada día se repite."),
    (0, "La vida parece no tener color ni sabor."),
    (0, "A veces, ni siquiera quiero levantarme."),
    (0, "Siento que estoy atrapado en este estado."),
    (0, "Cada día es igual de oscuro que el anterior."),
    (0, "Las lágrimas están ahí, incluso si no las dejo salir."),
    (0, "Es como si estuviera en una nube gris constante."),
    (0, "Nada me hace feliz, solo veo lo negativo."),
    (0, "Me siento como si estuviera en un agujero sin salida."),
    (0, "No tengo ganas de ver a nadie, prefiero estar solo."),
    (0, "Es como si estuviera en una pesadilla sin fin."),
    (0, "No tengo fuerzas para sonreír ni para fingir."),
    (0, "Siento que la vida me ha dejado sin energía."),
    (0, "A veces, quisiera desaparecer."),
    (0, "Es como si un manto oscuro me cubriera todo el tiempo."),
    (0, "No encuentro razones para continuar."),
    (0, "La tristeza se ha vuelto una constante en mi vida."),
    (0, "No hay nada que me motive a seguir."),
    (0, "Cada día se siente igual de pesado."),
    (0, "Siento que no encajo en ningún lugar."),
    (0, "La vida parece vacía y sin sentido."),
    (0, "El futuro parece oscuro y sin esperanza."),
    (0, "No encuentro nada que me haga feliz."),
    (0, "Cada día es una lucha por encontrar algo positivo."),
    (0, "La tristeza es abrumadora, difícil de soportar."),
    (0, "No veo una salida a este estado."),
    (0, "El dolor en mi corazón es constante y profundo."),
    (0, "Cada día parece más difícil que el anterior."),
    (0, "La soledad se siente pesada y difícil de llevar."),
    (0, "Es como si el tiempo se hubiera detenido en este dolor."),
    (0, "No tengo interés en el futuro, todo parece oscuro."),
    (0, "Es un ciclo sin fin de tristeza y desesperanza."),
    (0, "No tengo energía ni ganas de hacer nada."),
    (0, "Me siento atrapado en este estado sin salida."),
    (0, "No encuentro consuelo en nada ni en nadie."),
    (0, "Cada día me pesa más que el anterior."),
    (0, "Siento que he perdido el rumbo de mi vida."),
    (0, "Nada parece mejorar, solo se siente peor."),
])
dataset.append((0, "Hoy me siento solo y sin ánimos de hacer nada."))
dataset.append((0, "Es un día gris, siento que nada me motiva."))
dataset.append((0, "Me siento vacío y sin esperanza, no veo el sentido."))
dataset.append((0, "La tristeza se ha apoderado de mí, todo parece oscuro."))
dataset.append((0, "Apenas tengo ganas de salir de la cama hoy."))
dataset.append((0, "Todo se siente difícil y agotador, sin energía para nada."))
dataset.append((0, "Siento que no encuentro mi lugar en el mundo."))
dataset.append((0, "Hoy no tengo motivación ni entusiasmo por nada."))
dataset.append((0, "Es un día pesado, lleno de pensamientos negativos."))
dataset.append((0, "La tristeza parece no querer irse, es difícil encontrar algo positivo."))
dataset.append((0, "No tengo fuerzas para enfrentar el día."))
dataset.append((0, "Siento que estoy atrapado en un ciclo sin fin."))
dataset.append((0, "Hoy es un día en el que todo parece ir mal."))
dataset.append((0, "El desánimo se ha convertido en mi constante compañero."))
dataset.append((0, "Nada parece tener sentido últimamente."))
dataset.append((0, "Estoy rodeado de oscuridad, no veo una salida."))
dataset.append((0, "Hoy siento que mi corazón está pesado de tristeza."))
dataset.append((0, "Nada de lo que hago parece aliviar esta sensación."))
dataset.append((0, "Me siento perdido y sin rumbo."))
dataset.append((0, "Hoy es uno de esos días donde la tristeza pesa mucho."))
dataset.append((0, "No tengo fuerzas para sonreír hoy."))
dataset.append((0, "Todo parece estar en mi contra."))
dataset.append((0, "Es un día en el que solo quiero desaparecer."))
dataset.append((0, "No tengo ganas de hablar con nadie."))
dataset.append((0, "Hoy me siento débil y sin esperanza."))
dataset.append((0, "La tristeza es como una sombra que no me deja."))
dataset.append((0, "Siento que estoy en un pozo sin fondo."))
dataset.append((0, "Nada parece mejorar, todo se siente igual de mal."))
dataset.append((0, "No sé cómo seguir adelante en días como hoy."))
dataset.append((0, "Es difícil encontrar algo que me haga sentir bien."))
dataset.append((0, "Cada día se siente más pesado que el anterior."))
dataset.append((0, "No veo luz al final de este túnel de tristeza."))
dataset.append((0, "Hoy no encuentro razón para estar contento."))
dataset.append((0, "La soledad se ha vuelto mi única compañía."))
dataset.append((0, "Me siento atrapado en esta tristeza."))
dataset.append((0, "Nada de lo que hago me ayuda a sentirme mejor."))
dataset.append((0, "Hoy es uno de esos días donde nada parece mejorar."))
dataset.append((0, "Me siento exhausto emocionalmente."))
dataset.append((0, "La tristeza se ha vuelto parte de mí hoy."))
dataset.append((0, "No tengo fuerzas para enfrentar el día."))
dataset.append((0, "Hoy todo parece más oscuro de lo normal."))
dataset.append((0, "Me siento incomprendido y solo."))
dataset.append((0, "La tristeza me consume lentamente."))
dataset.append((0, "Es difícil encontrar algo positivo hoy."))
dataset.append((0, "No tengo ganas de seguir adelante."))
dataset.append((0, "Hoy todo me parece sin sentido."))
dataset.append((0, "Siento que estoy perdido en un mar de tristeza."))
dataset.append((0, "Hoy me cuesta encontrar una razón para sonreír."))
dataset.append((0, "La vida parece ser solo tristeza últimamente."))
dataset.append((0, "No sé si esta sensación algún día desaparecerá."))
dataset.append((0, "Es como si el peso del mundo estuviera sobre mí."))
dataset.append((0, "No tengo ganas de hablar o hacer nada hoy."))
dataset.append((0, "Cada día es más difícil soportar este sentimiento."))
dataset.append((0, "Hoy siento que no pertenezco a ningún lugar."))
dataset.append((0, "La tristeza es mi única compañía hoy."))
dataset.append((0, "Todo parece más sombrío y pesado."))
dataset.append((0, "Estoy cansado de sentirme así, sin esperanzas."))
dataset.append((0, "No encuentro nada que me haga sentir mejor."))
dataset.append((0, "Cada momento se siente más doloroso que el anterior."))
dataset.append((0, "Hoy la tristeza se siente más profunda que nunca."))

# Textos que representan el estado de ánimo "Melancólico"

dataset.append((1, "Extraño los viejos tiempos, hay cierta nostalgia en el aire."))
dataset.append((1, "Me encuentro pensativo, recordando momentos del pasado."))
dataset.append((1, "A veces, el silencio me hace reflexionar sobre lo que fue."))
dataset.append((1, "La melancolía se ha apoderado de mí, pero no me siento triste."))
dataset.append((1, "Hoy siento un vacío inexplicable, algo me falta."))
dataset.append((1, "Recuerdo con cariño esos momentos que ya no volverán."))
dataset.append((1, "La nostalgia es una compañía silenciosa hoy."))
dataset.append((1, "Me encuentro mirando fotos antiguas y recordando buenos tiempos."))
dataset.append((1, "Los días pasan y siento que algo se perdió en el camino."))
dataset.append((1, "Estoy atrapado entre recuerdos de un tiempo mejor."))
dataset.append((1, "Hoy los recuerdos me llenan de una calma melancólica."))
dataset.append((1, "Pensar en el pasado me trae una sensación de paz."))
dataset.append((1, "Me siento nostálgico por momentos que ya se fueron."))
dataset.append((1, "Es uno de esos días en los que el pasado pesa más."))
dataset.append((1, "Hoy recuerdo con cariño lo que alguna vez tuve."))
dataset.append((1, "Es una tristeza suave, una melancolía persistente."))
dataset.append((1, "Me encuentro revisando cartas y recordando buenos tiempos."))
dataset.append((1, "El pasado parece más vivo que el presente hoy."))
dataset.append((1, "Cada recuerdo me llena de un suave sentimiento de nostalgia."))
dataset.append((1, "Los recuerdos del pasado me traen una tristeza leve."))
dataset.append((1, "Hoy todo me recuerda tiempos mejores."))
dataset.append((1, "No estoy triste, solo lleno de melancolía."))
dataset.append((1, "Pensar en esos días me llena de un extraño vacío."))
dataset.append((1, "Es como si los recuerdos fueran más reales que el presente."))
dataset.append((1, "La nostalgia es una dulce compañía hoy."))
dataset.append((1, "Recuerdo las risas, los momentos que ya no volverán."))
dataset.append((1, "Estoy perdido en pensamientos de lo que fue."))
dataset.append((1, "Los días pasan, pero los recuerdos se quedan."))
dataset.append((1, "Hoy estoy envuelto en pensamientos de otros tiempos."))
dataset.append((1, "El silencio trae recuerdos de un tiempo más simple."))
dataset.append((1, "Me siento atrapado en un mundo de recuerdos."))
dataset.append((1, "Hoy todo me hace recordar tiempos pasados."))
dataset.append((1, "Es una melancolía tranquila la que siento hoy."))
dataset.append((1, "Estoy rodeado de recuerdos que no se irán."))
dataset.append((1, "La nostalgia me acompaña como una sombra."))
dataset.append((1, "Hoy me pierdo en pensamientos de lo que alguna vez fue."))
dataset.append((1, "La paz del pasado me inunda, aunque no es real."))
dataset.append((1, "Es un día lleno de recuerdos que no volverán."))
dataset.append((1, "Estoy rodeado de un aire de nostalgia."))
dataset.append((1, "Recuerdo esos días y siento una calma triste."))
dataset.append((1, "Cada momento parece estar teñido de melancolía hoy."))
dataset.append((1, "La nostalgia es un eco constante en mi mente hoy."))
dataset.append((1, "Hoy no puedo evitar recordar aquellos tiempos."))
dataset.append((1, "Los recuerdos me llenan de una paz melancólica."))
dataset.append((1, "Hoy los pensamientos del pasado son mi única compañía."))
dataset.append((1, "No estoy triste, solo pensativo y nostálgico."))
dataset.append((1, "Es un día en el que me pierdo en los recuerdos."))
dataset.append((1, "Los días de antes parecen más cercanos hoy."))
dataset.append((1, "Me encuentro revisitando momentos de otro tiempo."))
dataset.append((1, "La melancolía se siente como un abrazo lejano hoy."))
dataset.append((1, "Recuerdo esos momentos y me invade una dulce tristeza."))
dataset.append((1, "Hoy, la nostalgia es una amiga silenciosa."))
dataset.append((1, "Cada cosa me recuerda algo que ya no está."))
dataset.append((1, "Hoy todo parece teñido de recuerdos."))
dataset.append((1, "Me siento atrapado en pensamientos de otro tiempo."))
dataset.append((1, "Hoy los recuerdos parecen estar más vivos que nunca."))
dataset.append((1, "Es como si el tiempo retrocediera en mi mente."))
dataset.append((1, "Estoy envuelto en una melancolía tranquila hoy."))
dataset.append((1, "Todo lo que veo me recuerda algo que extraño."))
dataset.append((1, "Es uno de esos días donde la nostalgia es mi compañera."))

dataset.extend([
    (1, "Hoy siento una nostalgia por los momentos pasados."),
    (1, "Recordar viejas amistades me trae una melancolía dulce."),
    (1, "Pienso en todo lo que ya no es y me siento melancólico."),
    (1, "Es una sensación de añoranza, sin tristeza real."),
    (1, "Siento como si un trozo de mi pasado me llamara."),
    (1, "Las memorias de esos tiempos me llenan de un extraño vacío."),
    (1, "A veces, las fotos antiguas traen una paz melancólica."),
    (1, "Recuerdo las risas de antes con cariño y algo de nostalgia."),
    (1, "La melancolía es suave y persistente hoy."),
    (1, "Me detengo a reflexionar en todo lo que fue."),
    (1, "Es un sentimiento tranquilo, pero un poco triste."),
    (1, "Hoy los recuerdos parecen más vivos que nunca."),
    (1, "Pienso en lo que pudo ser y nunca fue."),
    (1, "Esos tiempos parecen lejanos pero a la vez cercanos."),
    (1, "Recuerdo canciones viejas y la época que representaban."),
    (1, "Los momentos compartidos regresan a mi mente hoy."),
    (1, "El pasado es un visitante constante esta tarde."),
    (1, "Hoy la nostalgia me hace compañía."),
    (1, "A veces, pienso en lo que dejé atrás."),
    (1, "La melancolía viene y va como un susurro."),
    (1, "Recuerdo lo bueno y lo malo con serenidad."),
    (1, "Es como si los recuerdos estuvieran a un paso de distancia."),
    (1, "Los momentos de antaño parecen tan lejanos."),
    (1, "Pienso en cómo era todo antes."),
    (1, "Los días pasados se sienten tan cercanos hoy."),
    (1, "A veces, deseo regresar a esos tiempos."),
    (1, "Recuerdo los rostros que solían acompañarme."),
    (1, "La melancolía es como una brisa suave."),
    (1, "Me encuentro atrapado entre la realidad y los recuerdos."),
    (1, "A veces me pregunto cómo sería si las cosas hubieran sido distintas."),
    (1, "Es un estado en el que me pierdo en pensamientos."),
    (1, "La nostalgia parece aflorar con cada pensamiento."),
    (1, "Recuerdo lugares que ya no frecuento."),
    (1, "Cada recuerdo trae un matiz de melancolía."),
    (1, "Es como si el pasado estuviera más presente hoy."),
    (1, "Los días de antaño parecen tan llenos de vida."),
    (1, "Siento que algo importante se ha perdido."),
    (1, "Cada canción me recuerda a una época diferente."),
    (1, "Hoy todo me recuerda a esos tiempos."),
    (1, "La nostalgia parece ser mi compañera en este momento."),
    (1, "Cada cosa pequeña me hace recordar algo."),
    (1, "La nostalgia me invade de manera suave."),
    (1, "Siento que algo importante se ha quedado atrás."),
    (1, "Recuerdo lugares y personas como si fuera ayer."),
    (1, "Todo me recuerda a esos momentos que ya no están."),
    (1, "Es como si el pasado estuviera más vivo en mi mente."),
    (1, "Me pierdo en los recuerdos de tiempos que ya no son."),
    (1, "Cada detalle me lleva de vuelta a esos momentos."),
    (1, "Pienso en esos tiempos con cariño y un toque de tristeza."),
    (1, "Recuerdo cosas que solía amar."),
    (1, "Hoy siento que el pasado me llama."),
    (1, "Es una calma nostálgica que no me deja ir."),
    (1, "Me encuentro pensando en todo lo que fue."),
    (1, "A veces, lo viejo parece mejor que lo nuevo."),
    (1, "Es un sentimiento sereno pero presente."),
    (1, "Pienso en lo que dejé atrás."),
    (1, "Me siento en paz con los recuerdos, aunque traigan nostalgia."),
    (1, "Hoy, el pasado se siente más real que nunca."),
])
dataset.append((1, "Extraño los viejos tiempos, hay cierta nostalgia en el aire."))
dataset.append((1, "Me encuentro pensativo, recordando momentos del pasado."))
dataset.append((1, "A veces, el silencio me hace reflexionar sobre lo que fue."))
dataset.append((1, "La melancolía se ha apoderado de mí, pero no me siento triste."))
dataset.append((1, "Hoy siento un vacío inexplicable, algo me falta."))
dataset.append((1, "Recuerdo con cariño esos momentos que ya no volverán."))
dataset.append((1, "La nostalgia es una compañía silenciosa hoy."))
dataset.append((1, "Me encuentro mirando fotos antiguas y recordando buenos tiempos."))
dataset.append((1, "Los días pasan y siento que algo se perdió en el camino."))
dataset.append((1, "Estoy atrapado entre recuerdos de un tiempo mejor."))

# Textos que representan el estado de ánimo "Neutral"
dataset.append((2, "No ha habido nada especial, solo un día común."))
dataset.append((2, "Hoy ha sido un día sin grandes eventos ni emociones."))
dataset.append((2, "Es un día más, sin cambios significativos."))
dataset.append((2, "Siento que todo está en su lugar, nada fuera de lo normal."))
dataset.append((2, "Hoy es simplemente un día más en la semana."))
dataset.append((2, "Las cosas están tranquilas y en equilibrio."))
dataset.append((2, "Sigo con la rutina, sin emociones fuertes."))
dataset.append((2, "Es un día sin altibajos, bastante normal."))
dataset.append((2, "Todo se siente calmado y estable."))
dataset.append((2, "No tengo mucho que decir sobre mi día, es como cualquier otro."))
dataset.append((2, "Hoy ha sido un día sencillo y sin sorpresas."))
dataset.append((2, "La rutina sigue y no hay nada de especial."))
dataset.append((2, "No hay nada emocionante ni aburrido, solo normalidad."))
dataset.append((2, "Es uno de esos días en los que nada sobresale."))
dataset.append((2, "Siento que todo está en equilibrio, sin emociones fuertes."))
dataset.append((2, "Hoy ha sido un día más, sin nada digno de destacar."))
dataset.append((2, "No tengo sentimientos fuertes, todo es bastante neutral."))
dataset.append((2, "Es un día tranquilo, sin altibajos."))
dataset.append((2, "Me siento en paz, sin grandes emociones."))
dataset.append((2, "Es un día común y corriente, sin nada particular."))
dataset.append((2, "Hoy no ha habido nada emocionante, todo sigue igual."))
dataset.append((2, "La estabilidad y la calma han sido constantes hoy."))
dataset.append((2, "Nada especial ha ocurrido hoy, todo está igual."))
dataset.append((2, "Es simplemente otro día que pasa sin novedades."))
dataset.append((2, "Me siento sereno, sin mucho que reportar."))
dataset.append((2, "Hoy no hay emociones fuertes, solo serenidad."))
dataset.append((2, "Es uno de esos días en los que todo está en orden."))
dataset.append((2, "La normalidad es la única constante de mi día."))
dataset.append((2, "No tengo grandes emociones hoy, solo calma."))
dataset.append((2, "Es un día en que nada ha cambiado, todo sigue igual."))
dataset.append((2, "No ha habido nada fuera de lo común."))
dataset.append((2, "La rutina me acompaña y todo parece normal."))
dataset.append((2, "Es un día más sin ningún evento importante."))
dataset.append((2, "Hoy todo ha sido bastante ordinario."))
dataset.append((2, "Sigo en la misma sintonía, sin grandes cambios."))
dataset.append((2, "Es un día igual a los anteriores, bastante neutral."))
dataset.append((2, "Hoy la rutina ha sido la protagonista."))
dataset.append((2, "No tengo emociones fuertes hoy, solo estabilidad."))
dataset.append((2, "Es un día sin sorpresas ni emociones intensas."))
dataset.append((2, "Siento que nada ha cambiado hoy, es un día cualquiera."))
dataset.append((2, "Es un día como cualquier otro, tranquilo y sin novedades."))
dataset.append((2, "Hoy todo sigue su curso natural, sin sobresaltos."))
dataset.append((2, "Me siento en calma, sin grandes emociones."))
dataset.append((2, "Es simplemente otro día, sin nada emocionante."))
dataset.append((2, "No tengo ningún sentimiento en particular hoy."))
dataset.append((2, "Todo está en orden, sin novedades importantes."))
dataset.append((2, "Hoy ha sido un día sin cambios, solo estabilidad."))
dataset.append((2, "Es un día más en mi vida, sin sorpresas."))
dataset.append((2, "No siento nada especial hoy, solo serenidad."))
dataset.append((2, "La calma es la protagonista de mi día."))
dataset.append((2, "Es un día de pura rutina y tranquilidad."))
dataset.append((2, "Siento que todo sigue igual, en calma."))
dataset.append((2, "Es simplemente un día sin grandes emociones."))
dataset.append((2, "Hoy no tengo ningún sentimiento particular."))
dataset.append((2, "Es uno de esos días de pura rutina y tranquilidad."))
dataset.append((2, "Hoy todo ha transcurrido en calma y paz."))
dataset.append((2, "Es un día sin altibajos, bastante equilibrado."))
dataset.append((2, "No tengo emociones fuertes, todo es bastante normal."))
dataset.append((2, "Es un día más, sin nada fuera de lo común."))
dataset.append((2, "Me siento tranquilo, sin mucho que reportar."))
dataset.append((2, "Hoy simplemente ha sido un día normal, nada especial."))
dataset.append((2, "No tengo mucho que decir, todo está en equilibrio."))
dataset.append((2, "Hoy no siento emociones fuertes, solo paz."))
dataset.append((2, "Me siento estable y calmado, sin sobresaltos."))
dataset.append((2, "Es un día como cualquier otro, nada fuera de lo normal."))
dataset.append((2, "No tengo ningún sentimiento particular hoy."))
dataset.append((2, "Mi día ha sido bastante ordinario y simple."))
dataset.append((2, "La rutina sigue y me siento bastante neutral."))

# Textos que representan el estado de ánimo "Alegre"

dataset.append((3, "Hoy me siento lleno de energía y felicidad."))
dataset.append((3, "La vida es increíble, todo parece perfecto."))
dataset.append((3, "Estoy agradecido y satisfecho con mi día."))
dataset.append((3, "Me siento en paz y optimista con lo que viene."))
dataset.append((3, "Estoy riendo y disfrutando cada momento."))
dataset.append((3, "Nada puede arruinar mi buen humor hoy."))
dataset.append((3, "El sol brilla y todo se siente maravilloso."))
dataset.append((3, "Me siento afortunado por tener a personas increíbles a mi alrededor."))
dataset.append((3, "Hoy todo parece estar alineado para que sea un gran día."))
dataset.append((3, "La música me pone de buen humor y me llena de energía."))
dataset.append((3, "Hoy me siento increíble, todo va bien."))
dataset.append((3, "La vida me sonríe, me siento feliz y agradecido."))
dataset.append((3, "Es un día lleno de luz y alegría."))
dataset.append((3, "Estoy disfrutando cada momento, me siento en paz."))
dataset.append((3, "La felicidad me llena, es un gran día."))
dataset.append((3, "Todo parece positivo y lleno de oportunidades."))
dataset.append((3, "Me siento afortunado por tener este día hermoso."))
dataset.append((3, "La vida es buena, estoy muy contento hoy."))
dataset.append((3, "Nada puede arruinar mi buen humor hoy."))
dataset.append((3, "Estoy rodeado de personas maravillosas."))
dataset.append((3, "Hoy todo parece fluir y salir bien."))
dataset.append((3, "La alegría se siente en cada cosa que hago."))
dataset.append((3, "Me siento feliz, es un día perfecto."))
dataset.append((3, "Todo parece estar alineado para que sea un gran día."))
dataset.append((3, "La energía positiva me rodea y me llena de alegría."))
dataset.append((3, "Me siento lleno de esperanza y optimismo."))
dataset.append((3, "Hoy todo parece encajar perfectamente."))
dataset.append((3, "Me siento en sintonía con todo, es un día feliz."))
dataset.append((3, "Todo va de maravilla, estoy en paz."))
dataset.append((3, "Es un día lleno de risas y buenos momentos."))
dataset.append((3, "Estoy agradecido por las cosas simples de la vida."))
dataset.append((3, "La felicidad es la constante de mi día."))
dataset.append((3, "Me siento lleno de energía y entusiasmo."))
dataset.append((3, "Todo lo que veo me hace sonreír hoy."))
dataset.append((3, "La vida me ha dado motivos para estar feliz."))
dataset.append((3, "Me siento pleno y agradecido por este día."))
dataset.append((3, "Hoy todo parece tener sentido."))
dataset.append((3, "El sol y el aire fresco me hacen sentir feliz."))
dataset.append((3, "La buena energía me rodea en cada paso."))
dataset.append((3, "Estoy disfrutando cada segundo del día."))
dataset.append((3, "El optimismo es la constante hoy."))
dataset.append((3, "Es un día lleno de sonrisas y buenas vibras."))
dataset.append((3, "Me siento bendecido de vivir este día."))
dataset.append((3, "Estoy de buen humor y lleno de energía positiva."))
dataset.append((3, "Siento que nada puede salir mal hoy."))
dataset.append((3, "Me siento alegre, todo es perfecto hoy."))
dataset.append((3, "Es un día lleno de amor y felicidad."))
dataset.append((3, "Me siento lleno de gratitud por todo."))
dataset.append((3, "La vida se siente tan simple y hermosa hoy."))
dataset.append((3, "Estoy disfrutando de las pequeñas cosas del día."))
dataset.append((3, "Siento que todo está en su lugar, es un gran día."))
dataset.append((3, "Estoy feliz y lleno de paz."))
dataset.append((3, "Hoy la felicidad me acompaña."))
dataset.append((3, "Todo es tan hermoso que me llena de alegría."))
dataset.append((3, "Es un día de paz y felicidad."))
dataset.append((3, "La energía del día es tan positiva que me siento increíble."))
dataset.append((3, "Hoy todo es perfecto, lleno de armonía."))
dataset.append((3, "La vida es buena y lo siento en cada paso."))
dataset.append((3, "Me siento bendecido, es un día maravilloso."))
dataset.append((3, "Es un día especial, lleno de buenos momentos."))
dataset.append((3, "La felicidad es la protagonista de mi día."))
dataset.append((3, "Estoy en paz con el mundo, es un gran día."))
dataset.append((3, "Hoy todo es perfecto y lleno de alegría."))
dataset.append((3, "Siento que todo tiene un propósito hoy."))
dataset.append((3, "Me siento en armonía con todo y todos."))
dataset.append((3, "Es un día lleno de energía positiva y paz."))
dataset.append((3, "Me siento rodeado de amor y buenos sentimientos."))
dataset.append((3, "Hoy todo se siente tan claro y perfecto."))
dataset.append((3, "Es un día feliz y lleno de buenas vibras."))
dataset.append((3, "Me siento bendecido de estar aquí hoy."))

# Textos que representan el estado de ánimo "Eufórico"
dataset.append((4, "Me siento en la cima del mundo, lleno de adrenalina."))
dataset.append((4, "No puedo contener la emoción, ¡es el mejor día de mi vida!"))
dataset.append((4, "Estoy tan feliz que quiero gritar al mundo mi alegría."))
dataset.append((4, "Estoy lleno de energía, ¡como si pudiera hacer cualquier cosa!"))
dataset.append((4, "Hoy es un día épico, todo está saliendo perfecto."))
dataset.append((4, "Siento una emoción indescriptible, es pura euforia."))
dataset.append((4, "No puedo dejar de sonreír, ¡me siento increíble!"))
dataset.append((4, "Hoy es uno de esos días que recordaré para siempre."))
dataset.append((4, "Es como si el universo estuviera conspirando a mi favor."))
dataset.append((4, "Estoy lleno de energía, ¡como si pudiera volar!"))
dataset.append((4, "¡Hoy es el mejor día de mi vida, me siento imparable!"))
dataset.append((4, "¡La energía que tengo hoy es increíble, nada puede detenerme!"))
dataset.append((4, "¡Todo es posible, estoy lleno de una emoción indescriptible!"))
dataset.append((4, "Siento una felicidad que no puedo contener, ¡es perfecto!"))
dataset.append((4, "¡Me siento en la cima del mundo, es como un sueño!"))
dataset.append((4, "¡Nunca me había sentido tan vivo, esto es espectacular!"))
dataset.append((4, "Estoy rebosante de alegría, ¡no puedo creer lo increíble que es este día!"))
dataset.append((4, "¡Siento que puedo lograr cualquier cosa, esto es pura adrenalina!"))
dataset.append((4, "¡Hoy todo es perfecto, estoy lleno de euforia y energía!"))
dataset.append((4, "¡Este es el día más increíble de todos, me siento pleno!"))
dataset.append((4, "¡La felicidad me inunda, estoy viviendo un sueño!"))
dataset.append((4, "¡No puedo parar de sonreír, todo es maravilloso!"))
dataset.append((4, "Siento una explosión de emociones, ¡es un día espectacular!"))
dataset.append((4, "¡Todo lo que quiero está sucediendo, esto es mágico!"))
dataset.append((4, "¡Estoy tan emocionado que siento que voy a explotar de felicidad!"))
dataset.append((4, "¡Es un día de pura adrenalina, nada puede detenerme!"))
dataset.append((4, "¡Todo lo que veo me hace feliz, es como un sueño!"))
dataset.append((4, "¡Hoy estoy en el punto más alto, nada me frena!"))
dataset.append((4, "¡Es un día lleno de sorpresas y cada una es increíble!"))
dataset.append((4, "¡No puedo creer lo feliz que estoy, esto es surrealista!"))
dataset.append((4, "¡Me siento tan afortunado y emocionado que no sé qué hacer!"))
dataset.append((4, "¡Este es un día épico, nunca había sentido tanta energía!"))
dataset.append((4, "¡La vida es tan maravillosa que me siento en otro mundo!"))
dataset.append((4, "¡Cada momento es perfecto, es como si estuviera en un sueño!"))
dataset.append((4, "¡No puedo pedir más, es el mejor día que podría tener!"))
dataset.append((4, "¡La emoción me desborda, hoy todo es posible!"))
dataset.append((4, "¡Me siento en la cima de la felicidad, es un momento único!"))
dataset.append((4, "¡Hoy es un día espectacular, lleno de pura euforia!"))
dataset.append((4, "¡Estoy extasiado de felicidad, esto es increíble!"))
dataset.append((4, "¡La emoción me hace sentir como si estuviera volando!"))
dataset.append((4, "¡No puedo parar de sonreír, la felicidad es imparable!"))
dataset.append((4, "¡Cada segundo es increíble, esto es pura magia!"))
dataset.append((4, "¡Estoy viviendo el momento perfecto, todo es ideal!"))
dataset.append((4, "¡La vida no puede ser mejor, estoy en el mejor estado de ánimo!"))
dataset.append((4, "¡Hoy estoy invencible, nada puede arruinar este momento!"))
dataset.append((4, "¡La euforia me invade, es el mejor día de todos!"))
dataset.append((4, "¡No puedo creer la emoción que siento, esto es perfecto!"))
dataset.append((4, "¡Hoy todo es un sueño hecho realidad, es inigualable!"))
dataset.append((4, "¡Siento una energía que me da vida, es inigualable!"))
dataset.append((4, "¡Nada puede apagar mi felicidad, esto es increíble!"))
dataset.append((4, "¡Este es el momento más feliz de mi vida, es como un sueño!"))
dataset.append((4, "¡Estoy lleno de emoción, cada segundo es perfecto!"))
dataset.append((4, "¡La vida me sonríe, es un día inmejorable!"))
dataset.append((4, "¡Es un día de pura adrenalina, nada puede detenerme!"))
dataset.append((4, "¡Estoy tan feliz que no puedo contener mi euforia!"))
dataset.append((4, "¡Es el día más espectacular que he vivido!"))
dataset.append((4, "¡La felicidad que siento es inexplicable, esto es único!"))
dataset.append((4, "¡No hay palabras para describir lo increíble que es este día!"))
dataset.append((4, "¡Es el día más especial, estoy rebosante de alegría!"))
dataset.append((4, "¡Hoy estoy tan emocionado que no puedo pensar en otra cosa!"))
dataset.append((4, "¡Este momento es tan perfecto que parece un sueño!"))
dataset.append((4, "¡Es un día mágico, lleno de pura euforia!"))
dataset.append((4, "¡Me siento imparable, este es un día increíble!"))
dataset.append((4, "¡Es la mejor experiencia, estoy lleno de energía!"))
dataset.append((4, "¡Cada instante es una explosión de emociones, es asombroso!"))

new_phrases = [
    # Frases neutras
    "Es un día aburrido sin nada especial",
    "Me da igual todo",
    "Estoy aburrido y sin mucho que hacer",
    "Hoy es solo un día más",
    "Todo parece normal y rutinario",
    "Es un día como cualquier otro",
    "No hay nada especial en este momento",
    "Estoy aquí, simplemente existiendo",
    "El día sigue su curso sin muchas sorpresas",

    # Frases alegres
    "Estoy muy feliz y activo.",
    "Me siento muy bien",
    "Hoy me siento lleno de energía y motivación",
    "Es un día increíble y lleno de posibilidades",
    "Siento que nada puede detenerme hoy",
    "Estoy agradecido por todas las cosas buenas en mi vida",
    "Me siento en paz y con mucha alegría",
    "Hoy todo parece brillar",
    "Estoy muy emocionado por lo que viene",

    # Frases tristes
    "Tengo ganas de llorar",
    "Me siento triste y solo",
    "Todo parece tan oscuro hoy",
    "Estoy pasando por un momento difícil",
    "Hoy no tengo fuerzas para nada",
    "El vacío es lo único que siento ahora",
    "Siento una tristeza profunda y constante",
    "Me pesa el corazón y el alma",
    "No tengo motivación para seguir adelante",

    # Frases melancólicas
    "Recuerdo momentos del pasado y me gustaría volver",
    "Siento nostalgia por tiempos que ya se fueron",
    "A veces me pregunto cómo sería si hubiera hecho las cosas diferente",
    "Echo de menos aquellos días de tranquilidad",
    "Las memorias del pasado me envuelven en melancolía",
    "Siento una suave tristeza por lo que ya no es",
    "Esos días lejanos me parecen irreales ahora",
    "Hay una paz triste en recordar lo que fue",
    "El pasado me trae una mezcla de felicidad y tristeza",

    # Frases eufóricas
    "¡No puedo creer lo bien que me siento hoy!",
    "¡Este es el mejor día de mi vida!",
    "Siento que puedo conquistar el mundo",
    "¡Estoy tan emocionado que quiero gritar de alegría!",
    "Todo es perfecto, no puedo pedir más",
    "¡Hoy me siento en la cima del mundo!",
    "La vida nunca ha sido tan increíble como ahora",
    "¡Estoy lleno de energía y quiero hacer de todo!",
    "Es como si el universo conspirara a mi favor",
    "¡Me siento imparable, como si nada pudiera salir mal!"
]
orden = ["Neutras", "Alegres", "Tristes", "Melancolicas", "Eufóricas"]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
true_labels = [2] * 9 + [3] * 9 + [0] * 9 + [1] * 9 + [4] * 10

#### Pruebas de modelos

Antes de comenzar las pruebas, definimos una función que nos devuelve las métricas.

In [ ]:
from sklearn.metrics import confusion_matrix
def calcular_metricas(true_labels, etiquetas_predichas):
  # Definimos las categorías únicas (asegurándonos de que están en el mismo orden en true_labels y etiquetas_predichas)
  labels_list = list(set(true_labels))

  # Creamos la matriz de confusión para comparar las etiquetas reales con las predicciones
  conf_matrix = confusion_matrix(true_labels, etiquetas_predichas, labels=labels_list)

  # Inicializamos un diccionario para almacenar las métricas por categoría
  metrics = {label: {'precision': 0, 'recall': 0, 'f1_score': 0} for label in labels_list}

  # Inicializamos variables para promedios de precisión, recall y f1-score
  f1_scores_sum = 0

  # Calculamos precisión, recall y f1-score para cada categoría
  for i, label in enumerate(labels_list):
      TP = conf_matrix[i, i]  # Verdaderos positivos
      FP = conf_matrix[:, i].sum() - TP  # Falsos positivos
      FN = conf_matrix[i, :].sum() - TP  # Falsos negativos

      # Calculamos precisión, recall y f1-score, con verificación para evitar divisiones por cero
      precision = TP / (TP + FP) if (TP + FP) > 0 else 0
      recall = TP / (TP + FN) if (TP + FN) > 0 else 0
      f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

      # Guardamos los resultados en el diccionario de métricas
      metrics[label]['precision'] = precision
      metrics[label]['recall'] = recall
      metrics[label]['f1_score'] = f1_score

      # Sumarizamos F1-scores para el cálculo del F1-score general
      f1_scores_sum += f1_score

  # Calculamos el F1-score general (macro F1-score) como el promedio de los F1-scores por categoría
  f1_score_general = f1_scores_sum / len(labels_list)

  # Calculamos el accuracy general
  accuracy = sum(conf_matrix[i, i] for i in range(len(labels_list))) / conf_matrix.sum()

  # Mostramos los resultados
  print(f"Accuracy general: {accuracy:.2f}")
  print(f"F1-Score general: {f1_score_general:.2f}")
  for label, scores in metrics.items():
      print(f"\nCategoría '{label}':")
      print(f"   Precisión: {scores['precision']:.2f}")
      print(f"   Recall: {scores['recall']:.2f}")
      print(f"   F1-Score: {scores['f1_score']:.2f}")

##### Modelo de Regresión Logística


In [ ]:
# Preparar X e y
X = [text.lower() for label, text in dataset]
y = [label for label, text in dataset]

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorización de los textos con eliminación de palabras vacías
vectorizer = TfidfVectorizer(stop_words=spanish_stop_words)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Creación y entrenamiento del modelo de Regresión Logística con multinomial
modelo_LR = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
modelo_LR.fit(X_train_vectorized, y_train)

# Evaluación del modelo de Regresión Logística
y_pred_LR = modelo_LR.predict(X_test_vectorized)
acc_LR = accuracy_score(y_test, y_pred_LR)
report_LR = classification_report(y_test, y_pred_LR, zero_division=1)

print("Precisión Regresión Logística:", acc_LR)
print("Reporte de clasificación Regresión Logística:\n", report_LR)


Precisión Regresión Logística: 0.7888888888888889
Reporte de clasificación Regresión Logística:
               precision    recall  f1-score   support

           0       0.86      0.81      0.83        31
           1       0.68      0.88      0.77        17
           2       0.86      0.86      0.86        14
           3       0.69      0.79      0.73        14
           4       0.89      0.57      0.70        14

    accuracy                           0.79        90
   macro avg       0.80      0.78      0.78        90
weighted avg       0.80      0.79      0.79        90



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
# Convertimos las frases a minúsculas
new_phrases = [frase.lower() for frase in new_phrases]

# Transformamos las nuevas frases usando el vectorizador que usamos para entrenar el modelo
nuevas_frases_vectorizadas = vectorizer.transform(new_phrases)

# Usamos el modelo entrenado para predecir las etiquetas de las nuevas frases
etiquetas_predichas = modelo_LR.predict(nuevas_frases_vectorizadas)

cont_orden = 0
# Imprimimos las etiquetas predichas
for i, etiqueta in enumerate(etiquetas_predichas):
    if(i % 9 == 0 and i < 44):
      print("----------------------------------------------------------")
      print(f'Frases {orden[cont_orden]}')
      print("----------------------------------------------------------")
      cont_orden += 1
    print(f"La frase '{new_phrases[i]}' pertenece a la categoría: {labels[etiqueta][1]}")


----------------------------------------------------------
Frases Neutras
----------------------------------------------------------
La frase 'es un día aburrido sin nada especial' pertenece a la categoría: Neutral
La frase 'me da igual todo' pertenece a la categoría: Triste
La frase 'estoy aburrido y sin mucho que hacer' pertenece a la categoría: Triste
La frase 'hoy es solo un día más' pertenece a la categoría: Neutral
La frase 'todo parece normal y rutinario' pertenece a la categoría: Triste
La frase 'es un día como cualquier otro' pertenece a la categoría: Neutral
La frase 'no hay nada especial en este momento' pertenece a la categoría: Melancólico
La frase 'estoy aquí, simplemente existiendo' pertenece a la categoría: Neutral
La frase 'el día sigue su curso sin muchas sorpresas' pertenece a la categoría: Neutral
----------------------------------------------------------
Frases Alegres
----------------------------------------------------------
La frase 'estoy muy feliz y activo.' p

In [ ]:
calcular_metricas(true_labels, etiquetas_predichas)

Accuracy general: 0.72
F1-Score general: 0.72

Categoría '0':
   Precisión: 0.50
   Recall: 0.67
   F1-Score: 0.57

Categoría '1':
   Precisión: 0.57
   Recall: 0.89
   F1-Score: 0.70

Categoría '2':
   Precisión: 1.00
   Recall: 0.56
   F1-Score: 0.71

Categoría '3':
   Precisión: 1.00
   Recall: 0.67
   F1-Score: 0.80

Categoría '4':
   Precisión: 0.89
   Recall: 0.80
   F1-Score: 0.84


##### Modelo de Regresión Logística con modelo preentrenado


In [ ]:
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Preparar X e y
X = [text.lower() for label, text in dataset]
y = [label for label, text in dataset]

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Obtenemos los embeddings de BERT para los conjuntos de entrenamiento y prueba
X_train_vectorized = model.encode(X_train)
X_test_vectorized = model.encode(X_test)

# Crear el pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.97)),
    ('classifier', LogisticRegression(max_iter=1000, solver='liblinear', C=1, penalty='l2'))
])

# Entrenar el pipeline
pipeline.fit(X_train_vectorized, y_train)

# Predecir y evaluar
y_pred_LR = pipeline.predict(X_test_vectorized)
acc_LR = accuracy_score(y_test, y_pred_LR)
report_LR = classification_report(y_test, y_pred_LR, zero_division=1)

print("Precisión con Pipeline:", acc_LR)
print("Reporte de clasificación con Pipeline:\n", report_LR)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Precisión con Pipeline: 0.8777777777777778
Reporte de clasificación con Pipeline:
               precision    recall  f1-score   support

           0       0.93      0.84      0.88        31
           1       0.88      0.88      0.88        17
           2       0.81      0.93      0.87        14
           3       0.79      0.79      0.79        14
           4       0.93      1.00      0.97        14

    accuracy                           0.88        90
   macro avg       0.87      0.89      0.88        90
weighted avg       0.88      0.88      0.88        90



In [ ]:
from sklearn.model_selection import cross_val_score

# Validación cruzada en el pipeline
scores = cross_val_score(pipeline, X_train_vectorized, y_train, cv=5, scoring='f1_macro')
print("F1_macro de validación cruzada:", scores.mean())


F1_macro de validación cruzada: 0.8254616831277485


In [ ]:
# Preprocesamiento y vectorización de las nuevas frases
new_phrases_lower = [text.lower() for text in new_phrases]
new_phrases_vectorized = model.encode(new_phrases_lower)  # Vectorización con el modelo preentrenado
# Usar el pipeline para hacer predicciones
new_predictions = pipeline.predict(new_phrases_vectorized)

cont_orden = 0
for i, etiqueta in enumerate(new_predictions):
    if(i % 9 == 0 and i < 44):
      print("----------------------------------------------------------")
      print(f'Frases {orden[cont_orden]}')
      print("----------------------------------------------------------")
      cont_orden += 1
    print(f"La frase '{new_phrases[i]}' pertenece a la categoría: {labels[etiqueta][1]}")

----------------------------------------------------------
Frases Neutras
----------------------------------------------------------
La frase 'es un día aburrido sin nada especial' pertenece a la categoría: Neutral
La frase 'me da igual todo' pertenece a la categoría: Alegre
La frase 'estoy aburrido y sin mucho que hacer' pertenece a la categoría: Neutral
La frase 'hoy es solo un día más' pertenece a la categoría: Triste
La frase 'todo parece normal y rutinario' pertenece a la categoría: Neutral
La frase 'es un día como cualquier otro' pertenece a la categoría: Triste
La frase 'no hay nada especial en este momento' pertenece a la categoría: Triste
La frase 'estoy aquí, simplemente existiendo' pertenece a la categoría: Neutral
La frase 'el día sigue su curso sin muchas sorpresas' pertenece a la categoría: Triste
----------------------------------------------------------
Frases Alegres
----------------------------------------------------------
La frase 'estoy muy feliz y activo.' pertene

In [ ]:
calcular_metricas(true_labels, new_predictions)

Accuracy general: 0.72
F1-Score general: 0.72

Categoría '0':
   Precisión: 0.47
   Recall: 0.78
   F1-Score: 0.58

Categoría '1':
   Precisión: 0.78
   Recall: 0.78
   F1-Score: 0.78

Categoría '2':
   Precisión: 1.00
   Recall: 0.44
   F1-Score: 0.62

Categoría '3':
   Precisión: 0.89
   Recall: 0.89
   F1-Score: 0.89

Categoría '4':
   Precisión: 0.78
   Recall: 0.70
   F1-Score: 0.74


##### Modelo con Random Forest

In [ ]:
# Preparar X e y
X = [text.lower() for label, text in dataset]
y = [label for label, text in dataset]

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Obtenemos los embeddings de BERT para los conjuntos de entrenamiento y prueba
X_train_vectorized = model.encode(X_train)
X_test_vectorized = model.encode(X_test)

# Crear y entrenar el modelo de Random Forest
modelo_RF = RandomForestClassifier(n_estimators=1000, random_state=42, max_features="sqrt", max_depth=15)  # puedes ajustar n_estimators
modelo_RF.fit(X_train_vectorized, y_train)

# Predicción con el modelo de Random Forest
y_pred_RF = modelo_RF.predict(X_test_vectorized)

# Evaluación del modelo de Random Forest
acc_RF = accuracy_score(y_test, y_pred_RF)
report_RF = classification_report(y_test, y_pred_RF, zero_division=1)

print("Precisión Random Forest:", acc_RF)
print("Reporte de clasificación Random Forest:\n", report_RF)


Precisión Random Forest: 0.7333333333333333
Reporte de clasificación Random Forest:
               precision    recall  f1-score   support

           0       0.81      0.68      0.74        31
           1       0.54      0.88      0.67        17
           2       0.78      0.50      0.61        14
           3       0.90      0.64      0.75        14
           4       0.82      1.00      0.90        14

    accuracy                           0.73        90
   macro avg       0.77      0.74      0.73        90
weighted avg       0.77      0.73      0.73        90



In [ ]:
# Preprocesamiento y vectorización de las nuevas frases
new_phrases_lower = [text.lower() for text in new_phrases]
new_phrases_vectorized = model.encode(new_phrases_lower)

# Haciendo predicciones con el modelo entrenado
new_predictions = modelo_RF.predict(new_phrases_vectorized)

# Mostrando las predicciones junto con las frases
cont_orden = 0
for i, etiqueta in enumerate(new_predictions):
    if(i % 9 == 0 and i < 44):
      print("----------------------------------------------------------")
      print(f'Frases {orden[cont_orden]}')
      print("----------------------------------------------------------")
      cont_orden += 1
    print(f"La frase '{new_phrases[i]}' pertenece a la categoría: {labels[etiqueta][1]}")

----------------------------------------------------------
Frases Neutras
----------------------------------------------------------
La frase 'es un día aburrido sin nada especial' pertenece a la categoría: Triste
La frase 'me da igual todo' pertenece a la categoría: Triste
La frase 'estoy aburrido y sin mucho que hacer' pertenece a la categoría: Melancólico
La frase 'hoy es solo un día más' pertenece a la categoría: Triste
La frase 'todo parece normal y rutinario' pertenece a la categoría: Neutral
La frase 'es un día como cualquier otro' pertenece a la categoría: Melancólico
La frase 'no hay nada especial en este momento' pertenece a la categoría: Triste
La frase 'estoy aquí, simplemente existiendo' pertenece a la categoría: Neutral
La frase 'el día sigue su curso sin muchas sorpresas' pertenece a la categoría: Melancólico
----------------------------------------------------------
Frases Alegres
----------------------------------------------------------
La frase 'estoy muy feliz y act

In [ ]:
calcular_metricas(true_labels, new_predictions)

Accuracy general: 0.54
F1-Score general: 0.54

Categoría '0':
   Precisión: 0.31
   Recall: 0.44
   F1-Score: 0.36

Categoría '1':
   Precisión: 0.41
   Recall: 0.78
   F1-Score: 0.54

Categoría '2':
   Precisión: 1.00
   Recall: 0.22
   F1-Score: 0.36

Categoría '3':
   Precisión: 0.83
   Recall: 0.56
   F1-Score: 0.67

Categoría '4':
   Precisión: 0.88
   Recall: 0.70
   F1-Score: 0.78


##### Modelo Multinomial Naive Bayes

In [ ]:
# Preparar X e y
X = [text.lower() for label, text in dataset]
y = [label for label, text in dataset]

# División del dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Obtenemos los embeddings de BERT para los conjuntos de entrenamiento y prueba
vectorizer = TfidfVectorizer()

# Ajustar y transformar el conjunto de entrenamiento
X_train_vectorized = vectorizer.fit_transform(X_train)

# Transformar el conjunto de prueba
X_test_vectorized = vectorizer.transform(X_test)

# Crear y entrenar el modelo Multinomial Naive Bayes
modelo_MNB = MultinomialNB(alpha=0.1)
modelo_MNB.fit(X_train_vectorized, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred_MNB = modelo_MNB.predict(X_test_vectorized)

# Evaluar el modelo
acc_MNB = accuracy_score(y_test, y_pred_MNB)
report_MNB = classification_report(y_test, y_pred_MNB, zero_division=1)

print("Precisión Multinomial Naive Bayes:", acc_MNB)
print("Reporte de clasificación Multinomial Naive Bayes:\n", report_MNB)


Precisión Multinomial Naive Bayes: 0.8555555555555555
Reporte de clasificación Multinomial Naive Bayes:
               precision    recall  f1-score   support

           0       0.96      0.77      0.86        31
           1       0.73      0.94      0.82        17
           2       0.93      1.00      0.97        14
           3       0.73      0.79      0.76        14
           4       0.92      0.86      0.89        14

    accuracy                           0.86        90
   macro avg       0.86      0.87      0.86        90
weighted avg       0.87      0.86      0.86        90



In [ ]:

# Preprocesamiento y vectorización de las nuevas frases
new_phrases_lower = [text.lower() for text in new_phrases]
new_phrases_vectorized =vectorizer.transform([text.lower() for text in new_phrases_lower])

# Haciendo predicciones con el modelo entrenado
new_predictions = modelo_MNB.predict(new_phrases_vectorized)

# Mostrando las predicciones junto con las frases
cont_orden = 0
for i, etiqueta in enumerate(new_predictions):
    if(i % 9 == 0 and i < 44):
      print("----------------------------------------------------------")
      print(f'Frases {orden[cont_orden]}')
      print("----------------------------------------------------------")
      cont_orden += 1
    print(f"La frase '{new_phrases[i]}' pertenece a la categoría: {labels[etiqueta][1]}")




----------------------------------------------------------
Frases Neutras
----------------------------------------------------------
La frase 'es un día aburrido sin nada especial' pertenece a la categoría: Neutral
La frase 'me da igual todo' pertenece a la categoría: Neutral
La frase 'estoy aburrido y sin mucho que hacer' pertenece a la categoría: Neutral
La frase 'hoy es solo un día más' pertenece a la categoría: Neutral
La frase 'todo parece normal y rutinario' pertenece a la categoría: Triste
La frase 'es un día como cualquier otro' pertenece a la categoría: Neutral
La frase 'no hay nada especial en este momento' pertenece a la categoría: Neutral
La frase 'estoy aquí, simplemente existiendo' pertenece a la categoría: Alegre
La frase 'el día sigue su curso sin muchas sorpresas' pertenece a la categoría: Neutral
----------------------------------------------------------
Frases Alegres
----------------------------------------------------------
La frase 'estoy muy feliz y activo.' pert

In [ ]:
calcular_metricas(true_labels, new_predictions)

Accuracy general: 0.85
F1-Score general: 0.84

Categoría '0':
   Precisión: 0.86
   Recall: 0.67
   F1-Score: 0.75

Categoría '1':
   Precisión: 0.82
   Recall: 1.00
   F1-Score: 0.90

Categoría '2':
   Precisión: 1.00
   Recall: 0.78
   F1-Score: 0.88

Categoría '3':
   Precisión: 0.78
   Recall: 0.78
   F1-Score: 0.78

Categoría '4':
   Precisión: 0.83
   Recall: 1.00
   F1-Score: 0.91


#### Decidimos quedarnos con el modelo Multinomial Naive Bayes ya que es el que mejor resultados ofrece en las nuevas predicciones observadas. Aparentemente los demás modelos estan teniendo overfitting. Un detalle curioso es que el valor bajo de alpha fue el que aumento accuracy y f1-score tanto en train como en test, pero supuestamente deberia aumentar el overfitting.

### 2. Ingreso de preferencias (inputs)

#### Determinar el estado de ánimo

In [131]:
animo_frase = {
    "Triste": "Hoy me siento un poco decaído, todo parece más difícil de lo normal. Necesito algo que me haga sentir mejor.",
    "Eufórico": "¡Estoy lleno de energía y alegría! Quiero hacer algo emocionante y lleno de adrenalina.",
    "Alegre": "Hoy estoy de buen ánimo, me siento feliz y quiero disfrutar de cosas que sigan elevando mi energía.",
    "Melancólico": "Hoy me siento algo nostálgico, recordando momentos pasados y buscando algo que me conecte con esos sentimientos.",
    "Neutral": "Hoy me siento en un punto intermedio, ni muy feliz ni triste. Simplemente estoy, buscando algo que me mantenga entretenido sin grandes altibajos."
}


In [132]:
def predecir_animo():
    animo_usuario = input("Ingrese cómo se siente: ")

    # Convertimos a minúsculas y vectorizamos
    animo_lower = animo_usuario.lower()
    vector_animo = vectorizer.transform([animo_lower])

    # Hacemos la predicción
    prediccion_animo = modelo_MNB.predict(vector_animo)

    # Buscamos la etiqueta en 'labels'
    for tupla in labels:
        if prediccion_animo[0] == tupla[0]:
            etiqueta = tupla[0]
            break

    print(f"La frase '{animo_usuario}' fue categorizada como estado de ánimo: {labels[etiqueta][1]}")
    animo_usuario = labels[etiqueta][1]
    return animo_usuario

# Llamamos a la función para probar
animo_usuario = predecir_animo()

# Usar el diccionario en el modelo
animo_usuario = animo_frase[animo_usuario]

Ingrese cómo se siente: muy feliz, muy sonriente, muy tranquilo
La frase 'muy feliz, muy sonriente, muy tranquilo' fue categorizada como estado de ánimo: Alegre


#### Temática que le gustaría explorar

In [133]:
tematica_usuario = input("Ingrese la temática que le gustaría explorar: ")

Ingrese la temática que le gustaría explorar: Me encanta perderme en historias de aventuras, con viajes y descubrimientos emocionantes


### 3. Búsqueda de opciones

#### Extracción de información necesaria de las películas.

In [ ]:
df_peliculas = pd.read_csv('IMDB-Movie-Data.csv')

df_peliculas = df_peliculas[['Title', 'Genre', 'Description']]
df_peliculas = df_peliculas.rename(columns = {'Description': 'description'})
df_peliculas

,Title,Genre,description
0,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...
1,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te..."
2,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...
3,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea..."
4,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...
...,...,...,...
995,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo..."
996,Hostel: Part II,Horror,Three American college students studying abroa...
997,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...
998,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...


#### Extracción de información necesaria de los juegos de mesa.

In [ ]:
df_jdm = pd.read_csv('/content/bgg_database.csv')

df_jdm = df_jdm[['game_name', 'description', 'categories', 'maxplayers']]

df_jdm

,game_name,description,categories,maxplayers
0,Brass: Birmingham,Brass: Birmingham is an economic strategy game...,"['Age of Reason', 'Economic', 'Industry / Manu...",4
1,Pandemic Legacy: Season 1,Pandemic Legacy is a co-operative campaign gam...,"['Environmental', 'Medical']",4
2,Gloomhaven,Gloomhaven is a game of Euro-inspired tactica...,"['Adventure', 'Exploration', 'Fantasy', 'Fight...",4
3,Ark Nova,"In Ark Nova, you will plan and design a modern...","['Animals', 'Economic', 'Environmental']",4
4,Twilight Imperium: Fourth Edition,Twilight Imperium (Fourth Edition) is a game o...,"['Civilization', 'Economic', 'Exploration', 'N...",6
...,...,...,...,...
995,Tiny Epic Dinosaurs,"In Tiny Epic Dinosaurs, breakthroughs in moder...","['Animals', 'Fantasy', 'Farming', 'Science Fic...",4
996,Kingsburg (Second Edition),The realm of Kingsburg is under attack! Monstr...,"['City Building', 'Dice', 'Fantasy', 'Medieval']",5
997,Condottiere,It is 13th century Italy. Trade is flourishing...,"['Bluffing', 'Card Game', 'Medieval', 'Negotia...",6
998,Qwirkle,The abstract game of Qwirkle consists of 108 w...,['Abstract Strategy'],4


#### Web Scraping para extraer información sobre 1000 libros más populares del proyecto Gutenberg.

In [ ]:
# URL de la página de los 1000 libros
url = 'https://www.gutenberg.org/browse/scores/top1000.php#books-last1'

# Hacer una solicitud a la página
response = requests.get(url)

# Comprobar que la solicitud fue exitosa
if response.status_code == 200:
    # Parsear el contenido de la página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrar todos los enlaces a los libros
    book_links = []
    for link in soup.find_all('a', href=True):
        if link['href'].startswith('/ebooks/'):
            book_links.append('https://www.gutenberg.org' + link['href'])
book_links = book_links[4:1004]
print(book_links)

['https://www.gutenberg.org/ebooks/84', 'https://www.gutenberg.org/ebooks/1342', 'https://www.gutenberg.org/ebooks/2701', 'https://www.gutenberg.org/ebooks/1513', 'https://www.gutenberg.org/ebooks/25558', 'https://www.gutenberg.org/ebooks/25344', 'https://www.gutenberg.org/ebooks/2542', 'https://www.gutenberg.org/ebooks/11', 'https://www.gutenberg.org/ebooks/100', 'https://www.gutenberg.org/ebooks/2641', 'https://www.gutenberg.org/ebooks/145', 'https://www.gutenberg.org/ebooks/46', 'https://www.gutenberg.org/ebooks/37106', 'https://www.gutenberg.org/ebooks/844', 'https://www.gutenberg.org/ebooks/64317', 'https://www.gutenberg.org/ebooks/67979', 'https://www.gutenberg.org/ebooks/16389', 'https://www.gutenberg.org/ebooks/174', 'https://www.gutenberg.org/ebooks/6593', 'https://www.gutenberg.org/ebooks/6761', 'https://www.gutenberg.org/ebooks/394', 'https://www.gutenberg.org/ebooks/5200', 'https://www.gutenberg.org/ebooks/2160', 'https://www.gutenberg.org/ebooks/4085', 'https://www.gutenbe

In [ ]:
books_data = []
for book_link in book_links:
    book_response = requests.get(book_link)
    if book_response.status_code == 200:
        book_soup = BeautifulSoup(book_response.text, 'html.parser')

        # Extraer título y autor
        title = book_soup.find('h1').text.strip()
        if "by" in title:
          title, author = title.rsplit(" by ", 1)
        else:
            author = "Desconocido"

        # Inicializar resumen
        summary = 'Resumen no disponible'

        # Buscar la tabla de clase "bibrec"
        bibrec_table = book_soup.find('table', class_='bibrec')
        if bibrec_table:
            # Buscar todas las filas en la tabla
            for row in bibrec_table.find_all('tr'):
                th = row.find('th')
                if th and th.text.strip() == 'Summary':
                    td = row.find('td')
                    if td:
                        summary = td.text.strip()
                        break

        # Almacenar la información en un diccionario
        if summary != 'Resumen no disponible':
          books_data.append({
              'title': title,
              'author': author,
              'summary': summary
          })
    else:
        print(f'Error al acceder a {book_link}: {book_response.status_code}')

# Ahora puedes hacer algo con los datos, por ejemplo, imprimirlos
for book in books_data:
    print(f'Título: {book["title"]}, Autor: {book["author"]}, Resumen: {book["summary"]}')

Título: Frankenstein; Or, The Modern Prometheus, Autor: Mary Wollstonecraft Shelley, Resumen: "Frankenstein; Or, The Modern Prometheus" by Mary Wollstonecraft Shelley is a novel written in the early 19th century. The story explores themes of ambition, the quest for knowledge, and the consequences of man's hubris through the experiences of Victor Frankenstein and the monstrous creation of his own making.   The opening of the book introduces Robert Walton, an ambitious explorer on a quest to discover new lands and knowledge in the icy regions of the Arctic. In his letters to his sister Margaret, he expresses both enthusiasm and the fear of isolation in his grand venture. As Walton's expedition progresses, he encounters a mysterious, emaciated stranger who has faced great suffering—furthering the intrigue of his narrative. This stranger ultimately reveals his tale of creation, loss, and the profound consequences of seeking knowledge that lies beyond human bounds. The narrative is set up i

In [ ]:
df_books = pd.DataFrame(books_data)

df_books.to_csv('df_books.csv', encoding='utf-8')

files.download('df_books.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_books = df_books.rename(columns = {'summary': 'description'})

### 4. Recomendaciones


In [134]:
# Inicializamos el modelo de embeddings
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Embedding del estado de ánimo y temática del usuario
estado_animo_embedding = model.encode(animo_usuario)
tematica_embedding = model.encode(tematica_usuario)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [135]:
def obtener_recomendaciones(estado_animo_embedding, tematica_embedding, dataset, embedding_column):
    dataset[embedding_column] = dataset[embedding_column].apply(lambda x: np.fromstring(x.strip("[]"), sep=' ') if isinstance(x, str) else x)
    # Calculamos la similitud de coseno para cada elemento en el dataset
    dataset['estado_animo_similitud'] = dataset[embedding_column].apply(lambda x: cosine_similarity([estado_animo_embedding], [x])[0][0])
    dataset['tematica_similitud'] = dataset[embedding_column].apply(lambda x: cosine_similarity([tematica_embedding], [x])[0][0])

    # Ponderación de las similitudes para obtener una similitud general
    dataset['similitud_total'] = 0.2 * dataset['estado_animo_similitud'] + 0.8 * dataset['tematica_similitud']
    # Ordenamos por similitud total y obtenemos las mejores recomendaciones
    recomendaciones = dataset.sort_values(by='similitud_total', ascending=False).head(5)
    print(recomendaciones['similitud_total'])
    return recomendaciones

# Recomendaciones de películas
recomendaciones_peliculas = obtener_recomendaciones(estado_animo_embedding, tematica_embedding, df_peliculas, 'description_embedding')
print("Recomendaciones de películas:")
print(recomendaciones_peliculas[['Title', 'Genre', 'description']])

# Recomendaciones de juegos de mesa
recomendaciones_juegos = obtener_recomendaciones(estado_animo_embedding, tematica_embedding, df_jdm, 'description_embedding')
print("\nRecomendaciones de juegos de mesa:")
print(recomendaciones_juegos[['game_name', 'categories', 'description']])

# Recomendaciones de libros
books_df = pd.DataFrame(books_data)
recomendaciones_libros = obtener_recomendaciones(estado_animo_embedding, tematica_embedding, books_df, 'summary_embedding')
print("\nRecomendaciones de libros:")
print(recomendaciones_libros[['title', 'author', 'summary']])


267    0.308559
299    0.270103
207    0.253141
813    0.249681
443    0.239521
Name: similitud_total, dtype: float64
Recomendaciones de películas:
                   Title                 Genre  \
267          Cloud Atlas          Drama,Sci-Fi   
299  There Will Be Blood         Drama,History   
207             Paterson  Comedy,Drama,Romance   
813    To Rome with Love        Comedy,Romance   
443       Blue Valentine         Drama,Romance   

                                           description  
267  An exploration of how the actions of individua...  
299  A story of family, religion, hatred, oil and m...  
207  A quiet observation of the triumphs and defeat...  
813  The lives of some visitors and residents of Ro...  
443  The relationship of a contemporary married cou...  
136    0.290903
920    0.255214
318    0.247982
85     0.245945
230    0.240987
Name: similitud_total, dtype: float64

Recomendaciones de juegos de mesa:
                     game_name  \
136                  

In [136]:
# Obtenemos embeddings para los textos de las descripciones en cada dataset
df_peliculas['description_embedding'] = df_peliculas['description'].apply(lambda x: model.encode(x))
df_jdm['description_embedding'] = df_jdm['description'].apply(lambda x: model.encode(x))
for book in books_data:
    book['summary_embedding'] = model.encode(book['summary'])


In [137]:
# Cargar el modelo GLiNER
modelo_gliner = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")
# Cargar el modelo para calcular embeddings multilingual
modelo_embeddings = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

def limpiar_texto(texto):
    texto = texto.lower()
    texto = unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode('utf-8')
    texto = re.sub(r'\W+', ' ', texto)
    return texto

def obtener_entidades(texto):
    """La función devuelve una lista de tuplas, donde cada tupla contiene el texto de la entidad y su etiqueta correspondiente."""
    texto = limpiar_texto(texto)

    labels = ["book", "location", "event", "person", "organization", "date", "time",
              "quantity", "product", "work of art", "language", "currency", "law", "facility"]

    entidades = modelo_gliner.predict_entities(texto, labels, threshold=0.4)
    return [(entidad["text"], entidad["label"]) for entidad in entidades]

def procesar_descripciones(data):
    # Obtener entidades
    data['Description_entities'] = data['description'].apply(obtener_entidades)

    embeddings = []

    # Obtener embedding
    for indice, fila in data.iterrows():
        emb_descripcion = modelo_embeddings.encode(fila['description'])
        embeddings.append(emb_descripcion)

    data['Description_embeddings'] = embeddings

    return data

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [138]:
df_peliculas = pd.read_csv('df_peliculas_entidad.csv')
df_jdm = pd.read_csv('df_jdm_entidad.csv')
df_books = pd.read_csv('df_books_entidad.csv')

In [ ]:
df_peliculas = procesar_descripciones(df_peliculas)
df_jdm = procesar_descripciones(df_jdm)
df_books = procesar_descripciones(df_books)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 446 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 394 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 408 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
/usr/local/lib/python3.10/dist-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 433 has been truncated to 384
  warnings.warn(f"Sentence of le

In [ ]:
df_books.to_csv('df_books.csv', encoding='utf-8')
files.download('df_books.csv')
df_jdm.to_csv('df_jdm.csv', encoding='utf-8')
files.download('df_jdm.csv')
df_peliculas.to_csv('df_peliculas.csv', encoding='utf-8')
files.download('df_peliculas.csv')

# Por lo mucho que tarda en realizarse este proceso guardamos los dataframes con los embbedings de las entidades

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [139]:
df_peliculas

,Unnamed: 0,Title,Genre,description,description_embedding,estado_animo_similitud,tematica_similitud,similitud_total,Description_entities,Description_embeddings
0,0,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,[-3.79823633e-02 -2.27691070e-03 -2.02692337e-...,0.099909,0.075767,0.080595,"[('intergalactic criminals', 'organization'), ...",[-2.91784815e-02 1.91560268e-01 -1.03249393e-...
1,1,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",[-4.28584777e-02 2.13632565e-02 -2.82521220e-...,0.026249,0.136380,0.114354,"[('team', 'organization'), ('distant moon', 'l...",[-1.24884389e-01 -4.43585739e-02 -2.11446404e-...
2,2,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,[ 1.17633333e-02 -3.18424143e-02 -1.66025721e-...,0.048440,0.209259,0.177096,"[('three girls', 'quantity'), ('man', 'person'...",[ 0.01779744 0.0538983 -0.14762166 -0.204962...
3,3,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",[ 2.29217112e-02 2.54714657e-02 5.57784177e-...,0.174467,0.388641,0.345806,"[('city of humanoid animals', 'location'), ('t...",[ 4.94425446e-01 -5.93847632e-02 -1.28133163e-...
4,4,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,[-8.47663060e-02 -1.49513558e-02 -1.26864284e-...,0.068992,0.030087,0.037868,"[('secret government agency', 'organization'),...",[-4.11478877e-01 2.46063694e-02 -4.56794232e-...
...,...,...,...,...,...,...,...,...,...,...
995,995,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",[-3.81939560e-02 -3.81066687e-02 -1.13781337e-...,0.086278,0.079089,0.080527,"[('rising investigators', 'organization'), ('s...",[-0.04789203 -0.0055514 0.0598441 0.040613...
996,996,Hostel: Part II,Horror,Three American college students studying abroa...,[ 5.93739450e-02 -9.27252248e-02 -6.90332130e-...,0.067242,0.083247,0.080046,"[('three american college students', 'person')...",[ 0.4808091 -0.01820599 -0.38124448 -0.521515...
997,997,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,[-5.36709577e-02 -2.53581926e-02 7.24388659e-...,0.079785,0.164344,0.147433,"[('romantic sparks', 'event'), ('two dance stu...",[ 1.06631527e-02 2.18147844e-01 2.65150249e-...
998,998,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,[-2.77204476e-02 4.41051507e-03 2.29930654e-...,0.105050,0.142728,0.135193,"[('pair of friends', 'person'), ('mission', 'e...",[ 0.09831724 0.23889609 -0.07967097 -0.149667...


In [140]:
df_jdm

,Unnamed: 0,game_name,description,categories,maxplayers,description_embedding,estado_animo_similitud,tematica_similitud,similitud_total,Description_entities,Description_embeddings
0,0,Brass: Birmingham,Brass: Birmingham is an economic strategy game...,"['Age of Reason', 'Economic', 'Industry / Manu...",4,[-5.08908406e-02 -8.70311540e-03 -3.13617028e-...,-0.022745,-0.143327,-0.119211,"[('brass birmingham', 'location'), ('martin wa...",[-1.91427171e-01 4.68948841e-01 -3.93081516e-...
1,1,Pandemic Legacy: Season 1,Pandemic Legacy is a co-operative campaign gam...,"['Environmental', 'Medical']",4,[ 2.82927305e-02 5.38452566e-02 -1.25769032e-...,0.136314,0.050057,0.067309,"[('pandemic', 'event'), ('world', 'location'),...",[ 1.19564049e-01 1.52945101e-01 -2.13119611e-...
2,2,Gloomhaven,Gloomhaven is a game of Euro-inspired tactica...,"['Adventure', 'Exploration', 'Fantasy', 'Fight...",4,[-1.37185240e-02 6.53882548e-02 -8.72832723e-...,0.081017,0.137741,0.126397,"[('gloomhaven', 'location'), ('players', 'pers...",[ 9.93047208e-02 -1.05188757e-01 -1.11760631e-...
3,3,Ark Nova,"In Ark Nova, you will plan and design a modern...","['Animals', 'Economic', 'Environmental']",4,[ 9.99899302e-03 4.93618473e-02 3.91022563e-...,0.022171,0.139986,0.116423,"[('ark nova', 'organization'), ('you', 'person...",[ 1.82816088e-01 4.95649874e-03 4.43276167e-...
4,4,Twilight Imperium: Fourth Edition,Twilight Imperium (Fourth Edition) is a game o...,"['Civilization', 'Economic', 'Exploration', 'N...",6,[-1.07175484e-02 8.46868753e-03 3.43503579e-...,0.130296,0.089563,0.097710,"[('twilight imperium', 'organization'), ('play...",[ 0.01118478 0.16435999 -0.18688601 -0.022776...
...,...,...,...,...,...,...,...,...,...,...,...
995,995,Tiny Epic Dinosaurs,"In Tiny Epic Dinosaurs, breakthroughs in moder...","['Animals', 'Fantasy', 'Farming', 'Science Fic...",4,[ 8.94727465e-03 5.34367338e-02 -1.29442150e-...,0.013302,0.083427,0.069402,"[('tiny epic dinosaurs', 'product'), ('rancher...",[-0.11546421 -0.18073446 0.18281454 -0.159387...
996,996,Kingsburg (Second Edition),The realm of Kingsburg is under attack! Monstr...,"['City Building', 'Dice', 'Fantasy', 'Medieval']",5,[ 1.82222191e-03 4.18009460e-02 -7.99152255e-...,0.032784,-0.001940,0.005005,"[('kingsburg', 'location'), ('the realm', 'loc...",[ 1.69562653e-01 1.49657324e-01 6.36599511e-...
997,997,Condottiere,It is 13th century Italy. Trade is flourishing...,"['Bluffing', 'Card Game', 'Medieval', 'Negotia...",6,[-7.73132369e-02 6.60007522e-02 -3.54418121e-...,0.067025,0.133870,0.120501,"[('13th century', 'time'), ('italy', 'location...",[ 5.05567119e-02 -2.10157901e-01 5.80327734e-...
998,998,Qwirkle,The abstract game of Qwirkle consists of 108 w...,['Abstract Strategy'],4,[-3.01586781e-02 2.08253022e-02 -6.67631254e-...,0.043016,0.013747,0.019601,"[('108 wooden blocks', 'quantity'), ('six diff...",[ 4.45967838e-02 5.07279765e-03 -1.49964124e-...


In [141]:
df_books

,Unnamed: 0,title,author,description,Description_entities,Description_embeddings
0,0,"Frankenstein; Or, The Modern Prometheus",Mary Wollstonecraft Shelley,"""Frankenstein; Or, The Modern Prometheus"" by M...","[('frankenstein or the modern prometheus', 'bo...",[-0.15921137 -0.11594973 -0.2094236 0.189335...
1,1,呻吟語,Kun Lü,"""呻吟語"" by Kun Lü is a philosophical treatise wr...","[('kun lu', 'person'), ('philosophical treatis...",[-6.45564198e-02 1.45035639e-01 -1.33199960e-...
2,2,Pride and Prejudice,Jane Austen,"""Pride and Prejudice"" by Jane Austen is a clas...","[('jane austen', 'person'), ('classic novel', ...",[-1.83390856e-01 8.71382840e-03 -2.40228117e-...
3,3,"Moby Dick; Or, The Whale",Herman Melville,"""Moby Dick; Or, The Whale"" by Herman Melville ...","[('moby dick or the whale', 'book'), ('herman ...",[ 0.01947979 0.23078172 -0.0364507 0.109149...
4,4,The Scarlet Letter,Nathaniel Hawthorne,"""The Scarlet Letter"" by Nathaniel Hawthorne is...","[('the scarlet letter', 'book'), ('nathaniel h...",[-2.34039187e-01 -1.17760956e-01 -1.20368987e-...
...,...,...,...,...,...,...
908,908,Now we are six,A. A. Milne,"""Now We Are Six"" by A. A. Milne is a collectio...","[('a milne', 'person'), ('early 20th century',...",[-9.35521722e-02 3.05579841e-01 1.05084836e-...
909,909,Fables de La Fontaine,Jean de La Fontaine,"""Fables de La Fontaine"" by Jean de La Fontaine...","[('fables de la fontaine', 'book'), ('jean de ...",[ 1.97994575e-01 1.49230719e-01 -3.27177942e-...
910,910,苦悶の欄,Earl Derr Biggers,"""苦悶の欄"" by Earl Derr Biggers is a novel written...","[('earl derr biggers', 'person'), ('novel', 'b...",[ 0.21936995 -0.02448611 -0.21312934 0.076076...
911,911,Alice in Wonderland,Alice Gerstenberg and Lewis Carroll,"""Alice in Wonderland"" by Alice Gerstenberg and...","[('alice', 'person'), ('wonderland', 'location...",[ 0.03166353 -0.17890832 -0.02869055 0.434775...


In [143]:
# Conseguimos las entidades de la tematica y el animo
entidades_usuario_tematica = obtener_entidades(tematica_usuario)
entidades_usuario_animo = obtener_entidades(animo_usuario)

# Convertir las entidades a embeddings utilizando el modelo de embeddings
entidades_usuario_tematica_embedding = modelo_embeddings.encode([entidad[0] for entidad in entidades_usuario_tematica])
entidades_usuario_animo_embedding = modelo_embeddings.encode([entidad[0] for entidad in entidades_usuario_animo])

def obtener_recomendaciones_final(dataset):
    recomendaciones = dataset.sort_values(by='similitud_total', ascending=False).head(5)
    print(recomendaciones['similitud_total'])
    return recomendaciones

def calcular_similitud_entidades(dataset, entidades_usuario_embedding):
    similitudes = []

    # Verificamos que entidades_usuario_embedding no esté vacío
    if entidades_usuario_embedding.size == 0:
        print("Error: el embedding de usuario está vacío.")
        return [0] * len(dataset)  # Retorna una lista de similitudes nulas si no hay embeddings de usuario

    for _, row in dataset.iterrows():
        # Verificamos si 'Description_embeddings' es un array o un string
        embedding_value = row['Description_embeddings']

        # Si es un string, convertir a un array
        if isinstance(embedding_value, str):
            embedding_value = np.fromstring(embedding_value.strip("[]"), sep=' ')

        # Verificamos si el embedding no está vacío y tiene la dimensión correcta
        if embedding_value.size > 0:
            # Ajustamos dimensión si es un array 1D
            entidad_embedding = embedding_value.reshape(1, -1)

            # Calculamos la similitud
            entidad_similitudes = cosine_similarity(entidad_embedding, entidades_usuario_embedding)
            similitud_promedio = np.mean(entidad_similitudes)
        else:
            similitud_promedio = 0  # Similitud nula si embedding está vacío

        similitudes.append(similitud_promedio)

    return similitudes

# Cálculo de similitudes de las entidades en los diferentes datasets
similitudes_entidades_tematica_peliculas = calcular_similitud_entidades(df_peliculas, entidades_usuario_tematica_embedding)
similitudes_entidades_animo_peliculas = calcular_similitud_entidades(df_peliculas, entidades_usuario_animo_embedding)

similitudes_entidades_tematica_jdm = calcular_similitud_entidades(df_jdm, entidades_usuario_tematica_embedding)
similitudes_entidades_animo_jdm = calcular_similitud_entidades(df_jdm, entidades_usuario_animo_embedding)

similitudes_entidades_tematica_libros = calcular_similitud_entidades(df_books, entidades_usuario_tematica_embedding)
similitudes_entidades_animo_libros = calcular_similitud_entidades(df_books, entidades_usuario_animo_embedding)

# Aplicamos la similitud total para cada dataset, ponderando las similitudes de las entidades
df_peliculas['similitud_total'] = (
    df_peliculas['similitud_total'] * 0.5 +
    0.5 * (0.7 * np.array(similitudes_entidades_tematica_peliculas) + 0.3 * np.array(similitudes_entidades_animo_peliculas))
)

df_jdm['similitud_total'] = (
    df_jdm['similitud_total'] * 0.5 +
    0.5 * (0.7 * np.array(similitudes_entidades_tematica_jdm) + 0.3 * np.array(similitudes_entidades_animo_jdm))
)

if 'similitud_total' not in df_books.columns:
    # Inicializa la columna si no existe
    df_books['similitud_total'] = 0

df_books['similitud_total'] = (
    df_books['similitud_total'] * 0.5 +
    0.5 * (0.7 * np.array(similitudes_entidades_tematica_libros) +
           0.3 * np.array(similitudes_entidades_animo_libros))
)

recomendaciones_peliculas = obtener_recomendaciones_final(df_peliculas)
print("Recomendaciones de películas:")
print(recomendaciones_peliculas[['Title', 'Genre', 'description']])

recomendaciones_juegos = obtener_recomendaciones_final(df_jdm)
print("\nRecomendaciones de juegos de mesa:")
print(recomendaciones_juegos[['game_name', 'categories', 'description']])

recomendaciones_libros = obtener_recomendaciones_final(df_books)
print("\nRecomendaciones de libros:")
print(recomendaciones_libros[['title', 'author', 'description']])

841    0.396881
321    0.342853
813    0.330347
721    0.303085
378    0.291100
Name: similitud_total, dtype: float64
Recomendaciones de películas:
                 Title                      Genre  \
841    A Bigger Splash             Drama,Thriller   
321           Lovesong                      Drama   
813  To Rome with Love             Comedy,Romance   
721               Wild  Adventure,Biography,Drama   
378     Swiss Army Man     Adventure,Comedy,Drama   

                                           description  
841  The vacation of a famous rock star and a filmm...  
321  The relationship between two friends deepens d...  
813  The lives of some visitors and residents of Ro...  
721  A chronicle of one woman's 1,100-mile solo hik...  
378  A hopeless man stranded on a deserted island b...  
764    0.394379
163    0.346660
204    0.324939
85     0.318482
178    0.306897
Name: similitud_total, dtype: float64

Recomendaciones de juegos de mesa:
                                     